In [8]:
import numpy as np
import picos as pic
import cvxopt as cvx
import random

typenum = random.randint(1,4)

defnum = random.randint(2,5)
attnum = random.randint(2,5)
DefenderRF = [[[random.randint(-5,6) for p in range(attnum)] for q in range(defnum)] for r in range(typenum)]
AttackerRF = [[[random.randint(-5,6) for p in range(attnum)] for q in range(defnum)] for r in range(typenum)]

Prob = [random.randint(1,10) for p in range(typenum)]
s = sum(Prob)
for i in range(0,len(Prob)):
    Prob[i]=Prob[i]/s

def DOBSS(AttackerRF,DefenderRF,Prob):
    prob = pic.Problem()
    l=len(DefenderRF[0])
    m=len(DefenderRF[0][0])
    tnum=len(DefenderRF)
    x = prob.add_variable('x',l,lower=0,upper=1)
    na = prob.add_variable('na',(tnum,m),'binary')
    v = prob.add_variable('v',tnum)
    rfa = pic.new_param('rfa',AttackerRF)
    rfd = pic.new_param('rfd',DefenderRF)
    pr = pic.new_param('pr',Prob)

    prob.add_constraint(pic.sum(x[i] for i in range(l))==1)
    for j in range(0,tnum):
        prob.add_constraint(pic.sum(na[j,i] for i in range(m))==1)

    for j in range(0,tnum):
        prob.add_list_of_constraints([v[j]-(x.T*rfa[j])[i]>0 for i in range(m)],'i','[m]')
        prob.add_list_of_constraints([v[j]-(x.T*rfa[j])[i]<1000000*(1-na[j,i]) for i in range(m)],'i','[m]')

    obj=0
    for j in range(0,tnum):
        obj=obj+pr[j]*x.T*rfd[j]*(na[j,:]).T

    #obj = pic.sum([pr[p]*x.T*rfd[p]*(na[p,:]).T for p in range(tnum)],'p','[tnum]')
    prob.set_objective('max',obj)
    prob.solve(verbose=0)
    return np.array(x),np.array(obj)

d0,rew0 = DOBSS(AttackerRF,DefenderRF,Prob)
print("Final Solution:\n")
print("Stackelberg Strategy:\n",d0)
print("Expected reward: ",rew0)

<ipython-input-8-429fd323f361>:23: DeprecationWarning: Problem.add_variable is deprecated: Variables can now be created independent of problems, and do not need to be added to any problem explicitly.
  x = prob.add_variable('x',l,lower=0,upper=1)
<ipython-input-8-429fd323f361>:24: DeprecationWarning: Problem.add_variable is deprecated: Variables can now be created independent of problems, and do not need to be added to any problem explicitly.
  na = prob.add_variable('na',(tnum,m),'binary')
<ipython-input-8-429fd323f361>:25: DeprecationWarning: Problem.add_variable is deprecated: Variables can now be created independent of problems, and do not need to be added to any problem explicitly.
  v = prob.add_variable('v',tnum)
<ipython-input-8-429fd323f361>:26: DeprecationWarning: new_param is deprecated: Use picos.Constant instead.
  rfa = pic.new_param('rfa',AttackerRF)
<ipython-input-8-429fd323f361>:27: DeprecationWarning: new_param is deprecated: Use picos.Constant instead.
  rfd = pic.ne

SolutionFailure: ignored

In [15]:

import numpy as np

def solve_bayesian_stackelberg_game(reward_matrix_leader, reward_matrix_follower, num_types):
    # first probabilities for the attcker are from a uniform dist
    initial_type_probs = np.ones(num_types) / num_types

    def leader_optimization(type_probs_follower):
        expected_rewards = np.dot(type_probs_follower, reward_matrix_follower)
        leader_strategy = np.argmax(expected_rewards)
        return leader_strategy

    def follower_optimization(type_probs_follower, leader_strategy):
        follower_utilities = reward_matrix_leader[:, leader_strategy]
        follower_strategy = np.argmax(follower_utilities)
        return follower_strategy

    # Iterative process
    max_iterations = 100
    tolerance = 1e-6
    type_probs_follower = initial_type_probs
    for _ in range(max_iterations):
        leader_strategy = leader_optimization(type_probs_follower)
        new_type_probs_follower = np.zeros(num_types)
        for i in range(num_types):
            follower_strategy = follower_optimization(type_probs_follower, leader_strategy)
            new_type_probs_follower[i] = np.dot(type_probs_follower, reward_matrix_follower[i, :])
        new_type_probs_follower /= np.sum(new_type_probs_follower)

        if np.max(np.abs(new_type_probs_follower - type_probs_follower)) < tolerance:
            break

        type_probs_follower = new_type_probs_follower

    return leader_strategy, type_probs_follower

# Example reward matrices
reward_matrix_leader = np.array([[70, 60], [80, 30]])
reward_matrix_follower = np.array([[30, 40], [20, 70]])
num_types = 2

leader_strategy, type_probs_follower = solve_bayesian_stackelberg_game(
    reward_matrix_leader, reward_matrix_follower, num_types
)

print("Leader's Strategy:", leader_strategy)
print("Follower's Type Probabilities:", type_probs_follower)

Leader's Strategy: 1
Follower's Type Probabilities: [0.42265027 0.57734973]


In [17]:
!pip install cyplex

ERROR: Could not find a version that satisfies the requirement cyplex (from versions: none)
ERROR: No matching distribution found for cyplex
